In [41]:
import tensorflow
import gym
import random
import atari_py
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy



from gym import Env
from gym.spaces import Discrete, Box
import random

import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [42]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38 + random.randint(-3,3)
        self.shower_lenght =60
        print(state)

    def step(self, action):
        self.state += (action -1)*2
        self.shower_lenght -=1


        if self.state >= 36 and self.state <= 40:
            reward =1
        else:
            reward = -1

        if self.shower_lenght <= 0:
            done = True
        else:
            done = False

        self.state += random.randint(-1,1)
        info = {}

        return self.state, reward, done, info

    def render(self):
        pass
    def reset(self):
        self.state = 38 + random.randint(-3,3)
        self.shower_lenght =60
        return self.state

    

In [43]:
env = ShowerEnv()

38


In [44]:
env.action_space.sample()

2

In [45]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} score:{}'.format(episode, score))
env.close()

Episode:1 score:-52
Episode:2 score:-44
Episode:3 score:-54
Episode:4 score:-38
Episode:5 score:18
Episode:6 score:-52
Episode:7 score:-22
Episode:8 score:-60
Episode:9 score:-42
Episode:10 score:-44


In [46]:
states = env.observation_space.shape
print(states)
actions = env.action_space.n

(1,)


In [47]:
def build_model(states, actions):
    model = Sequential()
    
    model.add(Dense(48, activation='relu', input_shape = states))
    
    model.add(Dense(21, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model


In [48]:
model = build_model(states, actions)



In [49]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    
    memory = SequentialMemory(limit =100, window_length =1)
    
    dqn = DQNAgent(model = model,
                   memory=memory,
                   policy=policy, 
                   
                   nb_actions = actions,
                   nb_steps_warmup =100,
                   target_model_update=1e-2
                  )
    return dqn

    
    

In [50]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-2), metrics=['mae'])
dqn.fit(env, nb_steps = 100000, visualize=False, verbose =1)

Training for 100000 steps ...
Interval 1 (0 steps performed)
 1187/10000 [==>...........................] - ETA: 50s - reward: -0.8147done, took 6.858 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))